# Search Stock on Screener: 


## 1.1 Search Function: 

In [1]:

def SearchStocksOnScreener(df, output_folder):
    import pandas as pd
    import requests as rq
    import time
    import json
    import re
    import datetime
    
    screener_stock_url = ""
    screener_stock_symbol = ""
    df_output = df.copy()

    for index, row in df.iterrows():
        nse_stock_symbol = row["Symbol"]
        print("Searching : ", nse_stock_symbol)
        # "https://www.screener.in/api/company/search/?q=<Stockname>&v=3&fts=1"
        search_url = "https://www.screener.in/api/company/search/?q=" +nse_stock_symbol+ "&v=3&fts=1"
        response = rq.get(search_url)
        time.sleep(1) # Seconds
        
        response_json = json.loads(response.text)
        
        if(response_json[0].get('id')==None):
            #print("Search Not Found")
            screener_stock_symbol = "<ErrorCode-1>"
            screener_stock_url = "<ErrorCode-1>"
        else:
            #print("Search Term Found: ",response_json[0].get('url'))
            url_text = response_json[0].get('url')  
            if m := re.match(r"\/company\/([A-Z0-9]+)\/.*", url_text):
                screener_stock_symbol = m.group(1)
                screener_stock_url = "https://www.screener.in" + url_text
            else:
                screener_stock_symbol = "<ErrorCode-2>"
                screener_stock_url = "<ErrorCode-2>"
                
        df_output.loc[index,'Screener Stock Symbol'] = screener_stock_symbol
        df_output.loc[index,'Screener Stock Url'] = screener_stock_url

    report_name = output_folder + "/" + "StockSymbolSearchReport-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".xlsx"
    writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
    df_output.to_excel(writer, sheet_name = 'StockSearch', index=False)
    writer.close()
    
    return report_name 

## 1.2 Instructions: 

### 1. Go to [NSE Website](https://www.nseindia.com/regulations/listing-compliance/nse-market-capitalisation-all-companies)
### 2. Download the Excel into Input folder.  
### 3. Edit the excel and format the Excel 


In [2]:
import pandas as pd 

input_excel = "../Input/MCAP28032024.xlsx"
input_excel = "../Input/MCAP-NSE.xlsx"
output_folder = "../Output/06-SearchStocks"

xls = pd.ExcelFile(input_excel)

df = pd.read_excel(xls, 'Sheet1')

df["Screener Stock Symbol"] = "<Null>"
df["Screener Stock Url"] = "<Null>"

## Remove invalid rows from tail of the DF manually
df.tail()

,Sr. No.,Symbol,Company Name,"Market capitalization as on March 28, 2024\n(In lakhs)",Screener Stock Symbol,Screener Stock Url
2376,2377.0,VISUINTL,Visu International Limited,"*Not available for trading as on March 28, 2024",<Null>,<Null>
2377,2378.0,VIVIMEDLAB,Vivimed Labs Limited,"*Not available for trading as on March 28, 2024",<Null>,<Null>
2378,2379.0,ZICOM,Zicom Electronic Security Systems Limited,"*Not available for trading as on March 28, 2024",<Null>,<Null>
2379,NaN,NaN,NaN,NaN,<Null>,<Null>
2380,NaN,Note: Kindly refer regulation 15 to 27 of SEBI...,NaN,NaN,<Null>,<Null>


In [5]:
# https://stackoverflow.com/a/49563326/1176573
n = 400
df_batch_of_nrows = [df[i:i+n] for i in range(0,df.shape[0],n)]
# [i.shape for i in df_batch_of_nrows]
print(len(df_batch_of_nrows))

6


### 4. Call Function:

In [ ]:
SearchStocksOnScreener(df_batch_of_nrows[0],output_folder)